In [ ]:
import sqlite3
import pandas as pd

# Kết nối tới file SQLite (tạo mới nếu chưa có)
conn = sqlite3.connect('/content/bank.db')
cursor = conn.cursor()


# Tạo bảng với các kiểu dữ liệu rõ ràng
cursor.execute("""
CREATE TABLE addresses (
    AddressID INT PRIMARY KEY,
    Street VARCHAR(100),
    City VARCHAR(100),
    Country VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE customer_types (
    CustomerTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE customers (
    CustomerID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    DateOfBirth DATE,
    AddressID INT,
    CustomerTypeID INT,
    FOREIGN KEY (AddressID) REFERENCES addresses(AddressID),
    FOREIGN KEY (CustomerTypeID) REFERENCES customer_types(CustomerTypeID)
);
""")

cursor.execute("""
CREATE TABLE account_types (
    AccountTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE account_statuses (
    AccountStatusID INT PRIMARY KEY,
    StatusName VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE accounts (
    AccountID INT PRIMARY KEY,
    CustomerID INT,
    AccountTypeID INT,
    AccountStatusID INT,
    Balance DECIMAL(15,2),
    OpeningDate TEXT,
    FOREIGN KEY (CustomerID) REFERENCES customers(CustomerID),
    FOREIGN KEY (AccountTypeID) REFERENCES account_types(AccountTypeID),
    FOREIGN KEY (AccountStatusID) REFERENCES account_statuses(AccountStatusID)
);
""")

cursor.execute("""
CREATE TABLE transaction_types (
    TransactionTypeID INT PRIMARY KEY,
    TypeName VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE branches (
    BranchID INT PRIMARY KEY,
    BranchName VARCHAR(100),
    AddressID INT,
    FOREIGN KEY (AddressID) REFERENCES addresses(AddressID)
);
""")

cursor.execute("""
CREATE TABLE transactions (
    TransactionID INT PRIMARY KEY,
    AccountID INT,
    BranchID INT,
    TransactionTypeID INT,
    Amount DECIMAL(15,2),
    Description VARCHAR(255),
    TransactionDate DATETIME,
    AccountOriginID INT,
    AccountDestinationID INT,
    FOREIGN KEY (AccountID) REFERENCES accounts(AccountID),
    FOREIGN KEY (BranchID) REFERENCES branches(BranchID),
    FOREIGN KEY (TransactionTypeID) REFERENCES transaction_types(TransactionTypeID)
);
""")

cursor.execute("""
CREATE TABLE loan_statuses (
    LoanStatusID INT PRIMARY KEY,
    StatusName VARCHAR(50)
);
""")

cursor.execute("""
CREATE TABLE loans (
    LoanID INT PRIMARY KEY,
    AccountID INT,
    LoanStatusID INT,
    Amount DECIMAL(15,2),
    InterestRate DECIMAL(5,2),
    StartDate DATE,
    EndDate DATE,
    FOREIGN KEY (AccountID) REFERENCES accounts(AccountID),
    FOREIGN KEY (LoanStatusID) REFERENCES loan_statuses(LoanStatusID)
);
""")

conn.commit()

In [65]:
# Đọc file CSV
accounts = pd.read_csv('/content/accounts.csv')
account_types = pd.read_csv('/content/account_types.csv')
account_statuses = pd.read_csv('/content/account_statuses.csv')

customers = pd.read_csv('/content/customers.csv')
customer_types = pd.read_csv('/content/customer_types.csv')

loans = pd.read_csv('/content/loans.csv')
loan_statuses = pd.read_csv('/content/loan_statuses.csv')

transactions = pd.read_csv('/content/transactions.csv')
transaction_types = pd.read_csv('/content/transaction_types.csv')

branches = pd.read_csv('/content/branches.csv')
addresses = pd.read_csv('/content/addresses.csv')

# Drop the tables if they exist
cursor.execute("DROP TABLE IF EXISTS accounts")
cursor.execute("DROP TABLE IF EXISTS account_types")
cursor.execute("DROP TABLE IF EXISTS account_statuses")

cursor.execute("DROP TABLE IF EXISTS customers")
cursor.execute("DROP TABLE IF EXISTS customer_types")

cursor.execute("DROP TABLE IF EXISTS loans")
cursor.execute("DROP TABLE IF EXISTS loan_statuses")

cursor.execute("DROP TABLE IF EXISTS transaction_types")
cursor.execute("DROP TABLE IF EXISTS transactions")

cursor.execute("DROP TABLE IF EXISTS branches")
cursor.execute("DROP TABLE IF EXISTS addresses")

# Nạp dữ liệu vào bảng
accounts.to_sql('accounts', conn, if_exists='append', index=False)
account_types.to_sql('account_types', conn, if_exists='append', index=False)
account_statuses.to_sql('account_statuses', conn, if_exists='append', index=False)

customers.to_sql('customers', conn, if_exists='append', index=False)
customer_types.to_sql('customer_types', conn, if_exists='append', index=False)

loans.to_sql('loans', conn, if_exists='append', index=False)
loan_statuses.to_sql('loan_statuses', conn, if_exists='append', index=False)

transactions.to_sql('transactions', conn, if_exists='append', index=False)
transaction_types.to_sql('transaction_types', conn, if_exists='append', index=False)

branches.to_sql('branches', conn, if_exists='append', index=False)
addresses.to_sql('addresses', conn, if_exists='append', index=False)

conn.commit()

In [61]:
demo = '''
SELECT *
FROM transactions
join branches using (branchid)
LIMIT 5'''
result = pd.read_sql_query(demo, conn)
print(result)

   TransactionID  AccountOriginID  AccountDestinationID  TransactionTypeID  \
0        3022681           201164                200868                  2   
1        3037846           200138                201402                  2   
2        3045293           201002                201180                  1   
3        3017397           201066                201144                  4   
4        3016750           200289                201413                  3   

    Amount      TransactionDate  BranchID        Description BranchName  \
0   855.17  2023-04-20 02:00:00        41  Transaction 22681  Branch 41   
1   806.20  2021-08-10 15:00:00        43  Transaction 37846  Branch 43   
2  1229.44  2020-08-16 03:00:00         5  Transaction 45293   Branch 5   
3  4441.60  2021-10-10 06:00:00        14  Transaction 17397  Branch 14   
4  2526.20  2022-07-28 00:00:00        37  Transaction 16750  Branch 37   

   AddressID  
0        173  
1        742  
2        796  
3        491  
4    

In [42]:
query = '''
SELECT ROUND(COUNT(DISTINCT customerid) * 100 / (SELECT COUNT(DISTINCT customerid) FROM accounts),5) AS Percentage
FROM accounts
WHERE strftime('%Y', openingdate) = '2018' '''
result = pd.read_sql_query(query, conn)
print(result)

   Percentage
0        38.0
